In [ ]:
import os, sys
import logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

logger.info("Application started successfully.")

In [ ]:
AWS_ACCESS_KEY = os.environ['AWS_ACCESS_KEY_ID']
AWS_SECRET_KEY  = os.environ['AWS_SECRET_ACCESS_KEY']
AWS_S3_ENDPOINT = os.environ["AWS_S3_ENDPOINT"]
FULL_PATH_TO_WAREHOUSE = os.environ['WAREHOUSE']


In [ ]:
# set pyspark configuration
from pyspark import SparkConf

conf = SparkConf() \
    .setAppName("read_bronze_adventure_works") \
    .setMaster("local[*]") \
    .set("spark.executor.memory", "4g") \
    .set("spark.driver.memory", "2g") \
    
# set MinIO config
conf.set("fs.s3a.access.key", AWS_ACCESS_KEY) 
conf.set("fs.s3a.secret.key", AWS_SECRET_KEY) 
conf.set("fs.s3a.endpoint", AWS_S3_ENDPOINT) 
conf.set("fs.s3a.connection.ssl.enabled", "false") 
conf.set("fs.s3a.path.style.access", "true") 

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

In [ ]:
# read config file
import toml

config_file = "notebooks/bronze/bronze_adventure_works.toml"
filepath = os.path.join(os.getcwd(), config_file)
with open(filepath, 'r') as f:
    config = toml.load(f)

# Access values
source_bucket = config['minio_source']['s3_bucket']
target_bucket = config['minio_target']['s3_bucket']
option = config['read_options']


In [ ]:
from pyspark.sql.functions import input_file_name, regexp_extract, current_timestamp, to_date, lit

for item in [config["source"][r] for r in config["source"] if config["source"][r]["enabled"]]:
    source_path = os.path.join(config["minio_source"]["s3_bucket"], item["source_path"])
    target_path = os.path.join(config["minio_target"]["s3_bucket"], "bronze", item["target_path"])

    df = spark.read.option("header", "true").csv(source_path)
    df = df.withColumn("file_path", input_file_name())
    df = df.withColumn("ingest_timestamp", current_timestamp())
    from datetime import datetime
    target_date = datetime.now().strftime("%Y-%m-%d")
    df = df.withColumn("ingest_date", lit(target_date))

    df.write.mode("overwrite") \
        .option("header", "true") \
        .partitionBy("ingest_date") \
        .csv(target_path)
    
    logger.info(f"Reprocessed data for {target_date} written to {target_path}")